# 🔧 RTL Agent Pipeline
**Natural Language → Gemini LLM → Verilog RTL → Icarus Simulation → Auto-Repair Loop**

```
Natural-Language / PDF Input
         ↓
   Gemini LLM (RTL + Testbench Generation)
         ↓
   ┌─────────────────────────────────┐
   │      AGENT CONTROL LAYER        │
   │  • Tracks iteration state       │
   │  • Decides compile vs repair    │
   │  • Maintains best candidate     │
   └──────────────┬──────────────────┘
                  ↓
         Icarus Verilog (iverilog + vvp)
                  ↓
   ┌─────────────────────────────────┐
   │     AGENT DECISION POLICY       │
   │  • Classify: pass / fail        │
   │  • Send errors back to Gemini   │
   │  • Accept or retry (max 3)      │
   └─────────────────────────────────┘
                  ↓
        Final RTL + Simulation Report
```

---
**Instructions:**
1. Run Cell 1 to install dependencies
2. Run Cell 2 to set your Gemini API key
3. Run Cell 3 to paste all pipeline code
4. Run Cell 4 and enter your hardware description (or upload a PDF)
5. Watch the agent iterate automatically!

In [ ]:
# ============================================================
# CELL 1: Install Dependencies
# ============================================================
!apt-get install -y iverilog -q
!pip install google-generativeai PyMuPDF Pillow -q
print('✅ All dependencies installed.')

Reading package lists...
Building dependency tree...
Reading state information...
Suggested packages:
  gtkwave
The following NEW packages will be installed:
  iverilog
0 upgraded, 1 newly installed, 0 to remove and 2 not upgraded.
Need to get 2,130 kB of archives.
After this operation, 6,749 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 iverilog amd64 11.0-1.1 [2,130 kB]
Fetched 2,130 kB in 0s (5,644 kB/s)
Selecting previously unselected package iverilog.
(Reading database ... 117540 files and directories currently installed.)
Preparing to unpack .../iverilog_11.0-1.1_amd64.deb ...
Unpacking iverilog (11.0-1.1) ...
Setting up iverilog (11.0-1.1) ...
Processing triggers for man-db (2.10.2-1) ...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.9/24.9 MB 75.8 MB/s eta 0:00:00
✅ All dependencies installed.


In [ ]:
# CELL 2 - Replace model names with these
from google.colab import userdata
import google.generativeai as genai

GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
genai.configure(api_key=GEMINI_API_KEY)
text_model   = genai.GenerativeModel('gemini-2.5-flash-lite')
vision_model = genai.GenerativeModel('gemini-2.5-flash-lite')
print('✅ Gemini configured.')

/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)


✅ Gemini configured.


In [ ]:
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(m.name)

models/gemini-2.5-flash
models/gemini-2.5-pro
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
models/gemini-2.5-flash-image
models/gemini-2.5-flash-lite-preview-09-2025
models/gemini-3-pro-preview
models/gemini-3-flash-preview
models/gemini-3.1-pro-preview
models/gemini-3.1-pro-preview-customtools
models/gemini-3-pro-image-preview
models/nano-banana-pro-preview
models/gemini-robotics-er-1.5-preview
models/gemini-2.5-computer-use-preview-10-2025
models/deep-research-pro-preview-12-2025


In [ ]:
# ============================================================
# CELL 3: Full Pipeline Code
# ============================================================
import subprocess
import json
import re
import os
import io
import fitz          # PyMuPDF
from PIL import Image
from pathlib import Path

WORK_DIR = Path('/content/rtl_agent')
WORK_DIR.mkdir(exist_ok=True)

MAX_RETRIES = 3

# ─────────────────────────────────────────────────────────────
# INPUT LAYER: PDF or Text
# ─────────────────────────────────────────────────────────────

def extract_from_pdf(pdf_path: str) -> str:
    """Extract everything from a PDF and let Gemini decide what's relevant."""
    print(f'📄 Reading PDF: {pdf_path}')
    doc = fitz.open(pdf_path)
    all_text = ''
    all_images = []

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        all_text += f'\n--- Page {page_num+1} ---\n'
        all_text += page.get_text()

        for img in page.get_images(full=True):
            xref = img[0]
            base_img = doc.extract_image(xref)
            pil_img = Image.open(io.BytesIO(base_img['image']))
            all_images.append(pil_img)

    print(f'  → Extracted {len(all_text)} chars, {len(all_images)} images')

    # Let Gemini summarize everything into a hardware description
    prompt_parts = [
        'You are an expert hardware design engineer. '
        'I am giving you raw content extracted from a document (text + images). '
        'Extract and summarize ALL hardware-relevant information: '
        'module names, port lists, truth tables, timing specs, logic descriptions, constraints. '
        'Output a clean, structured hardware specification that can be used to write Verilog RTL.\n\n'
        f'RAW TEXT:\n{all_text}'
    ] + all_images

    response = vision_model.generate_content(prompt_parts)
    hw_spec = response.text
    print('  → Gemini distilled PDF into hardware spec.')
    return hw_spec


def get_hardware_description() -> str:
    """Prompt user: type description OR upload a PDF."""
    print('\n' + '='*60)
    print('RTL AGENT PIPELINE - Input')
    print('='*60)
    print('Options:')
    print('  [1] Type / paste a hardware description')
    print('  [2] Provide a PDF file path')
    choice = input('\nChoice (1 or 2): ').strip()

    if choice == '2':
        pdf_path = input('PDF path (e.g. /content/spec.pdf): ').strip()
        return extract_from_pdf(pdf_path)
    else:
        print('\nPaste your hardware description below.')
        print('(Type END on a new line when done)\n')
        lines = []
        while True:
            line = input()
            if line.strip().upper() == 'END':
                break
            lines.append(line)
        return '\n'.join(lines)


# ─────────────────────────────────────────────────────────────
# LLM LAYER: Gemini calls
# ─────────────────────────────────────────────────────────────

def gemini_generate_rtl(hw_description: str) -> dict:
    """Ask Gemini to generate RTL + Testbench from a hardware description."""
    prompt = f"""You are an expert RTL design engineer using SystemVerilog/Verilog.

Given this hardware description, generate:
1. A synthesizable RTL module (DUT)
2. A self-contained Icarus Verilog-compatible testbench that:
   - Instantiates the DUT
   - Applies all meaningful input combinations
   - Uses $display to print results
   - Ends with $finish
   - Uses only Icarus-compatible constructs (no $fatal, no assertions)

HARDWARE DESCRIPTION:
{hw_description}

Respond ONLY in this exact JSON format:
{{
  "module_name": "<top module name>",
  "rtl_code": "<full Verilog/SV code for DUT>",
  "testbench_code": "<full Verilog/SV testbench code>",
  "explanation": "<brief explanation of the design>"
}}

Output ONLY the JSON. No markdown, no backticks."""

    response = text_model.generate_content(prompt)
    raw = response.text.strip()
    # Strip any accidental markdown fences
    raw = re.sub(r'^```[a-z]*\n?', '', raw, flags=re.MULTILINE)
    raw = re.sub(r'```$', '', raw, flags=re.MULTILINE)
    return json.loads(raw.strip())


def gemini_repair_rtl(hw_description: str, rtl_code: str, tb_code: str,
                      compile_result: dict, run_result: dict, attempt: int) -> dict:
    """Ask Gemini to fix RTL/TB based on compile and simulation errors."""
    compile_summary = (
        f"Return code: {compile_result['returncode']}\n"
        f"STDERR:\n{compile_result['stderr']}\n"
        f"STDOUT:\n{compile_result['stdout']}"
    )
    sim_summary = (
        f"Return code: {run_result['returncode']}\n"
        f"STDOUT:\n{run_result['stdout']}\n"
        f"STDERR:\n{run_result['stderr']}"
    ) if run_result else 'Simulation did not run (compile failed).'

    prompt = f"""You are an expert RTL debug engineer.

You generated Verilog RTL and a testbench for this hardware description:
{hw_description}

This is attempt {attempt} of 3. The simulation failed. Fix ONLY what is broken.
Do NOT rewrite from scratch unless absolutely necessary.
Focus on Icarus Verilog compatibility.

CURRENT RTL (DUT):
{rtl_code}

CURRENT TESTBENCH:
{tb_code}

COMPILE RESULT:
{compile_summary}

SIMULATION RESULT:
{sim_summary}

Common Icarus issues to check:
- Use `wire` not `logic` for module ports if compatibility issues
- No `$fatal`, use `$display + $finish` instead
- Timescale directive must be before module definition
- File I/O: $fopen/$fclose are fine
- No SystemVerilog-only constructs unless using -g2012 flag

Respond ONLY in this exact JSON format:
{{
  "module_name": "<top module name>",
  "rtl_code": "<fixed Verilog/SV code for DUT>",
  "testbench_code": "<fixed Verilog/SV testbench code>",
  "changes_made": "<bullet list of what you changed and why>"
}}

Output ONLY the JSON. No markdown, no backticks."""

    response = text_model.generate_content(prompt)
    raw = response.text.strip()
    raw = re.sub(r'^```[a-z]*\n?', '', raw, flags=re.MULTILINE)
    raw = re.sub(r'```$', '', raw, flags=re.MULTILINE)
    return json.loads(raw.strip())


# ─────────────────────────────────────────────────────────────
# TOOL EXECUTION LAYER: Icarus Verilog
# ─────────────────────────────────────────────────────────────

def run_cmd(cmd: list, cwd=None, timeout=60) -> dict:
    """Run a shell command and return structured result."""
    result = subprocess.run(
        cmd, cwd=str(cwd) if cwd else None,
        text=True, stdout=subprocess.PIPE,
        stderr=subprocess.PIPE, timeout=timeout
    )
    return {
        'cmd': ' '.join(cmd),
        'returncode': result.returncode,
        'stdout': result.stdout.strip(),
        'stderr': result.stderr.strip(),
    }


def write_and_compile(rtl_code: str, tb_code: str, module_name: str) -> tuple:
    """Write Verilog files and compile with Icarus."""
    dut_file = WORK_DIR / f'{module_name}.sv'
    tb_file  = WORK_DIR / f'tb_{module_name}.sv'
    sim_out  = WORK_DIR / 'sim.out'

    dut_file.write_text(rtl_code)
    tb_file.write_text(tb_code)

    compile_result = run_cmd(
        ['iverilog', '-g2012', '-o', str(sim_out),
         dut_file.name, tb_file.name],
        cwd=WORK_DIR
    )
    return compile_result, sim_out


def run_simulation(sim_out: Path) -> dict:
    """Run compiled simulation binary."""
    return run_cmd(['vvp', sim_out.name], cwd=WORK_DIR)



# ─────────────────────────────────────────────────────────────
# AGENT CONTROL + DECISION LAYER
# ─────────────────────────────────────────────────────────────

def print_banner(msg: str, char='='):
    print(f'\n{char*60}')
    print(f'  {msg}')
    print(f'{char*60}')


def agent_classify(compile_result: dict, run_result: dict) -> str:
    """
    Classify the current state.
    Returns: 'PASS', 'COMPILE_FAIL', or 'SIM_FAIL'
    """
    if compile_result['returncode'] != 0:
        return 'COMPILE_FAIL'
    if run_result and run_result['returncode'] != 0:
        return 'SIM_FAIL'
    return 'PASS'


def run_rtl_agent(hw_description: str):
    """Main agent loop."""
    print_banner('AGENT CONTROL LAYER - Starting')

    # Agent state
    state = {
        'best_candidate': None,
        'iteration': 0,
        'status': 'INIT',
        'history': []
    }

    rtl_code = tb_code = module_name = None

    for attempt in range(1, MAX_RETRIES + 1):
        state['iteration'] = attempt
        print_banner(f'ITERATION {attempt} / {MAX_RETRIES}', char='-')

        # ── Step 1: Generate or Repair RTL ──────────────────────
        if attempt == 1:
            print('🤖 Gemini: Generating initial RTL + Testbench...')
            try:
                result = gemini_generate_rtl(hw_description)
                rtl_code     = result['rtl_code']
                tb_code      = result['testbench_code']
                module_name  = result['module_name']
                print(f'  → Module: {module_name}')
                print(f'  → Explanation: {result.get("explanation", "")[:200]}')
            except Exception as e:
                print(f'❌ Gemini generation failed: {e}')
                print('  Raw response:', result if 'result' in dir() else 'N/A')
                break
        else:
            print(f'🔧 Gemini: Repairing RTL (attempt {attempt})...')
            prev = state['history'][-1]
            try:
                result = gemini_repair_rtl(
                    hw_description, rtl_code, tb_code,
                    prev['compile_result'], prev['run_result'], attempt
                )
                rtl_code    = result['rtl_code']
                tb_code     = result['testbench_code']
                module_name = result.get('module_name', module_name)
                print(f'  → Changes: {result.get("changes_made", "")[:300]}')
            except Exception as e:
                print(f'❌ Gemini repair failed: {e}')
                break

        # ── Step 2: Write & Compile ──────────────────────────────
        print(f'\n⚙️  TOOL: iverilog compile...')
        compile_result, sim_out = write_and_compile(rtl_code, tb_code, module_name)
        print(f'   Return code: {compile_result["returncode"]}')
        if compile_result['stderr']:
            print(f'   Stderr: {compile_result["stderr"][:500]}')

        # ── Step 3: Simulate (if compile passed) ─────────────────
        run_result = None
        if compile_result['returncode'] == 0:
            print(f'\n⚙️  TOOL: vvp simulation...')
            run_result = run_simulation(sim_out)
            print(f'   Return code: {run_result["returncode"]}')
            print(f'   Output:\n{run_result["stdout"][:1000]}')
            if run_result['stderr']:
                print(f'   Stderr: {run_result["stderr"][:500]}')

        # ── Step 4: Agent Decision Policy ────────────────────────
        status = agent_classify(compile_result, run_result)
        state['status'] = status
        state['history'].append({
            'attempt': attempt,
            'status': status,
            'compile_result': compile_result,
            'run_result': run_result,
            'rtl_code': rtl_code,
            'tb_code': tb_code,
        })

        print(f'\n📊 AGENT DECISION: {status}')

        if status == 'PASS':
            state['best_candidate'] = state['history'][-1]
            print('✅ All constraints satisfied. Terminating loop.')
            break
        elif attempt == MAX_RETRIES:
            print(f'⛔ Max retries ({MAX_RETRIES}) reached. Stopping.')
            # Save best candidate if any compile passed
            for h in reversed(state['history']):
                if h['compile_result']['returncode'] == 0:
                    state['best_candidate'] = h
                    break
        else:
            print(f'🔄 Sending errors back to Gemini for repair...')

    # ── Final Report ─────────────────────────────────────────────
    print_banner('FINAL REPORT')
    final_status = state['status']
    print(f'Status:      {final_status}')
    print(f'Iterations:  {state["iteration"]} / {MAX_RETRIES}')

    if state['best_candidate']:
        best = state['best_candidate']
        print(f'Best result: Attempt {best["attempt"]} ({best["status"]})')
        print(f'\n--- Final RTL Code ---')
        print(best['rtl_code'])
        print(f'\n--- Final Testbench ---')
        print(best['tb_code'])
        # Save final files
        (WORK_DIR / 'final_dut.sv').write_text(best['rtl_code'])
        (WORK_DIR / 'final_tb.sv').write_text(best['tb_code'])
        print(f'\n💾 Files saved to: {WORK_DIR}/final_dut.sv and final_tb.sv')
    else:
        print('❌ No passing candidate found.')

    # Full feedback packet
    feedback = {
        'final_status': final_status,
        'total_iterations': state['iteration'],
        'history': [
            {
                'attempt': h['attempt'],
                'status': h['status'],
                'compile_returncode': h['compile_result']['returncode'],
                'sim_returncode': h['run_result']['returncode'] if h['run_result'] else None,
            }
            for h in state['history']
        ]
    }
    print(f'\n--- Feedback Packet (JSON) ---')
    print(json.dumps(feedback, indent=2))
    return state


print('✅ Pipeline code loaded. Run the next cell to start.')

✅ Pipeline code loaded. Run the next cell to start.


In [ ]:
# NEW CELL - Run this RIGHT NOW before anything else
import google.generativeai as genai
from google.colab import userdata

genai.configure(api_key=userdata.get('GEMINI_API_KEY'))

print("Models available to your API key:")
for m in genai.list_models():
    if 'generateContent' in m.supported_generation_methods:
        print(m.name)

Models available to your API key:
models/gemini-2.5-flash
models/gemini-2.5-pro
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.5-flash-preview-tts
models/gemini-2.5-pro-preview-tts
models/gemma-3-1b-it
models/gemma-3-4b-it
models/gemma-3-12b-it
models/gemma-3-27b-it
models/gemma-3n-e4b-it
models/gemma-3n-e2b-it
models/gemini-flash-latest
models/gemini-flash-lite-latest
models/gemini-pro-latest
models/gemini-2.5-flash-lite
models/gemini-2.5-flash-image
models/gemini-2.5-flash-lite-preview-09-2025
models/gemini-3-pro-preview
models/gemini-3-flash-preview
models/gemini-3.1-pro-preview
models/gemini-3.1-pro-preview-customtools
models/gemini-3-pro-image-preview
models/nano-banana-pro-preview
models/gemini-robotics-er-1.5-preview
models/gemini-2.5-computer-use-preview-10-2025
models/deep-research-pro-preview-12-2025


In [ ]:
import google.generativeai
print(google.generativeai.__version__)

0.8.6


In [ ]:
!pip install -q --upgrade google-generativeai

In [ ]:
# ============================================================
# CELL 3b: PDF Upload + Extraction (add this before Cell 4)
# ============================================================
import fitz          # PyMuPDF
import io
import base64
from PIL import Image
from google.colab import files

def upload_and_extract_pdf() -> str:
    """
    1. Shows Colab upload button
    2. Extracts all text + images from uploaded PDF
    3. Sends everything to Gemini to distill into a hardware spec
    """
    print("📎 Click 'Choose Files' to upload your hardware spec PDF...")
    uploaded = files.upload()  # Opens Colab file picker

    if not uploaded:
        print("❌ No file uploaded.")
        return None

    # Get the uploaded file (take first if multiple)
    filename = list(uploaded.keys())[0]
    pdf_bytes = uploaded[filename]
    print(f"✅ Uploaded: {filename} ({len(pdf_bytes):,} bytes)")

    # ── Extract text + images from PDF ──────────────────────
    print("\n📄 Extracting text and images...")
    doc = fitz.open(stream=pdf_bytes, filetype="pdf")

    all_text = ""
    all_images = []

    for page_num in range(len(doc)):
        page = doc.load_page(page_num)

        # Extract text
        page_text = page.get_text()
        all_text += f"\n--- Page {page_num + 1} ---\n{page_text}"

        # Extract images
        for img in page.get_images(full=True):
            xref = img[0]
            try:
                base_img = doc.extract_image(xref)
                pil_img = Image.open(io.BytesIO(base_img["image"]))
                # Convert to RGB if needed (handles CMYK, RGBA etc.)
                if pil_img.mode not in ("RGB", "L"):
                    pil_img = pil_img.convert("RGB")
                all_images.append(pil_img)
            except Exception as e:
                print(f"  ⚠️  Skipped image on page {page_num+1}: {e}")

    print(f"  → {len(doc)} pages, {len(all_text):,} chars, {len(all_images)} images extracted")

    # ── Send everything to Gemini ────────────────────────────
    print("\n🤖 Gemini: Analyzing PDF content...")

    prompt = (
        "You are an expert hardware design engineer.\n"
        "I am giving you raw content extracted from a hardware specification document "
        "(text pages + circuit diagrams, truth tables, timing diagrams as images).\n\n"
        "Your job:\n"
        "1. Read ALL text and analyze ALL images carefully\n"
        "2. Extract every hardware-relevant detail: module names, port names, "
        "bit widths, truth tables, logic equations, timing constraints, "
        "interface specs, and any behavioral descriptions\n"
        "3. Output a clean, structured hardware specification formatted like this:\n\n"
        "MODULE NAME: <name>\n"
        "DESCRIPTION: <what it does>\n"
        "INPUTS: <list with bit widths>\n"
        "OUTPUTS: <list with bit widths>\n"
        "BEHAVIOR: <detailed logic description, truth tables, equations>\n"
        "CONSTRAINTS: <any timing or structural requirements>\n\n"
        "Be thorough — this spec will be used to generate Verilog RTL automatically.\n\n"
        f"EXTRACTED TEXT:\n{all_text}"
    )

    # Build content list: prompt text + all images
    content_parts = [prompt] + all_images

    try:
        response = vision_model.generate_content(content_parts)
        hw_spec = response.text
        print("  → Gemini distilled PDF into hardware specification ✅")
        print("\n" + "="*60)
        print("EXTRACTED HARDWARE SPECIFICATION:")
        print("="*60)
        print(hw_spec)
        print("="*60)
        return hw_spec
    except Exception as e:
        print(f"❌ Gemini extraction failed: {e}")
        print("  Falling back to raw text only...")
        return all_text.strip()


# ── Updated get_hardware_description with PDF option ────────
def get_hardware_description() -> str:
    """Choose between typing description or uploading PDF."""
    print("\n" + "="*60)
    print("RTL AGENT PIPELINE — Input")
    print("="*60)
    print("[1] Type / paste a hardware description")
    print("[2] Upload a PDF spec sheet")
    choice = input("\nChoice (1 or 2): ").strip()

    if choice == "2":
        return upload_and_extract_pdf()
    else:
        print("\nPaste your hardware description below.")
        print("Type END on a new line when done.\n")
        lines = []
        while True:
            line = input()
            if line.strip().upper() == "END":
                break
            lines.append(line)
        return "\n".join(lines)

print("✅ PDF input cell loaded. Now run Cell 4 to start.")

✅ PDF input cell loaded. Now run Cell 4 to start.


In [ ]:
# ============================================================
# CELL 4: Run the Agent (Text OR PDF)
# ============================================================
from google.colab import files
import fitz, io
from PIL import Image

print("="*60)
print("RTL AGENT PIPELINE — Choose Input Method")
print("="*60)
print("[1] Type / paste a hardware description")
print("[2] Upload a PDF spec sheet")
choice = input("\nChoice (1 or 2): ").strip()

if choice == "2":
    # ── PDF Upload ──────────────────────────────────────────
    print("\n📎 Click 'Choose Files' to upload your PDF...")
    uploaded = files.upload()
    filename = list(uploaded.keys())[0]
    pdf_bytes = uploaded[filename]
    print(f"✅ Uploaded: {filename}")

    # Extract text + images
    doc = fitz.open(stream=pdf_bytes, filetype="pdf")
    all_text = ""
    all_images = []
    for page_num in range(len(doc)):
        page = doc.load_page(page_num)
        all_text += f"\n--- Page {page_num+1} ---\n{page.get_text()}"
        for img in page.get_images(full=True):
            try:
                base_img = doc.extract_image(img[0])
                pil_img = Image.open(io.BytesIO(base_img["image"])).convert("RGB")
                all_images.append(pil_img)
            except:
                pass
    print(f"  → {len(doc)} pages, {len(all_images)} images extracted")

    # Gemini distills PDF into hardware spec
    print("🤖 Gemini analyzing PDF...")
    prompt = (
        "You are a hardware design engineer. Extract all hardware-relevant details "
        "from this document (text + diagrams): module name, ports, bit widths, "
        "truth tables, logic behavior, constraints. Output a clean structured spec "
        "that can be used to write Verilog RTL.\n\nEXTRACTED TEXT:\n" + all_text
    )
    response = vision_model.generate_content([prompt] + all_images)
    hw_description = response.text
    print("✅ Hardware spec extracted!")

else:
    # ── Manual Text Input ───────────────────────────────────
    print("\nPaste your hardware description below.")
    print("Type END on a new line when done.\n")
    lines = []
    while True:
        line = input()
        if line.strip().upper() == "END":
            break
        lines.append(line)
    hw_description = "\n".join(lines)

# ── Show what we got ────────────────────────────────────────
print(f"\n📝 Hardware Description ({len(hw_description)} chars):")
print("-"*60)
print(hw_description[:500] + ("..." if len(hw_description) > 500 else ""))
print("-"*60)

# ── Run the Agent ───────────────────────────────────────────
final_state = run_rtl_agent(hw_description)

RTL AGENT PIPELINE — Choose Input Method
[1] Type / paste a hardware description
[2] Upload a PDF spec sheet

Choice (1 or 2): 2

📎 Click 'Choose Files' to upload your PDF...


Saving Fast_Area_amp_Energy_Efficient_Supergate_Design_With_Multi-Output_amp_Multi-Functional_CDM_Cells 2.pdf to Fast_Area_amp_Energy_Efficient_Supergate_Design_With_Multi-Output_amp_Multi-Functional_CDM_Cells 2.pdf
✅ Uploaded: Fast_Area_amp_Energy_Efficient_Supergate_Design_With_Multi-Output_amp_Multi-Functional_CDM_Cells 2.pdf
  → 16 pages, 56 images extracted
🤖 Gemini analyzing PDF...
✅ Hardware spec extracted!

📝 Hardware Description (12219 chars):
------------------------------------------------------------
This document describes a Cell Design Methodology for Supergate (CDM-SG) design. The core idea is to create flexible and efficient cells that can implement multiple functions.

Here's a structured specification for Verilog RTL based on the extracted hardware-relevant details:

## CDM-SG Cell Specification

### Module Name: `cdm_sg_cell`

### Description:

This module implements a flexible supergate cell based on the Cell Design Methodology (CDM) proposed in the document. It is 

In [ ]:


# ============================================================
# CELL 5 (Optional): View saved files
# ============================================================
from pathlib import Path
WORK_DIR = Path('/content/rtl_agent')

print('=== final_dut.sv ===')
print((WORK_DIR / 'final_dut.sv').read_text() if (WORK_DIR / 'final_dut.sv').exists() else 'Not found')
print('\n=== final_tb.sv ===')
print((WORK_DIR / 'final_tb.sv').read_text() if (WORK_DIR / 'final_tb.sv').exists() else 'Not found')

=== final_dut.sv ===
Not found

=== final_tb.sv ===
Not found


In [ ]:
# ============================================================
# CELL 6 (Optional): Re-run simulation on saved files manually
# ============================================================
import subprocess
from pathlib import Path
WORK_DIR = Path('/content/rtl_agent')

dut = WORK_DIR / 'final_dut.sv'
tb  = WORK_DIR / 'final_tb.sv'
out = WORK_DIR / 'manual_sim.out'

# Compile
r = subprocess.run(
    ['iverilog', '-g2012', '-o', str(out), dut.name, tb.name],
    cwd=str(WORK_DIR), capture_output=True, text=True
)
print('Compile:', 'OK' if r.returncode == 0 else 'FAILED')
if r.stderr: print(r.stderr)

# Simulate
if r.returncode == 0:
    r2 = subprocess.run(['vvp', out.name], cwd=str(WORK_DIR), capture_output=True, text=True)
    print('\nSimulation Output:')
    print(r2.stdout)

Compile: FAILED
final_dut.sv: No such file or directory
No top level modules, and no -s option.

